In [1]:
import autoschema
help(autoschema)

Help on package autoschema:

NAME
    autoschema - # Package

PACKAGE CONTENTS
    autoschema
    util

FILE
    /Users/matthewharris/Documents/GItHub Repos/autoschema/venv/lib/python3.10/site-packages/autoschema/__init__.py


